Objective is to use census and epa keys to combine population data. The census will provide the population
and the EPA will provide the air quality data.
I would like to break the population (total population, population density) down by counties.

In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from census import Census
from us import states
import gmaps

# EPA API Key
from epakey import epa_key

# Census & gmaps API Keys
from censuskey import (census_key)
c = Census(census_key)

# Configure gmaps
#gmaps.configure(api_key=gkey)

In [47]:
# Run Census Search to retrieve data on all counties in all states
# The c.acs5.get is from the census wrapping in a github repo
census_data = c.acs5.get(("NAME", "B05012_001E"), {"for": "county:*"}, year = 2016)

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B05012_001E": "Population", "NAME": "Name", "county": "County"})
#census_pd_mn = census_pd.filter(like = "Minnesota")
census_pd_mn = census_pd[census_pd["Name"].str.contains("Minnesota")]
census_pd_mn2 = pd.concat([census_pd_mn, census_pd_mn["Name"].str.split(', ', expand=True)], axis=1)
census_pd_mn2 = census_pd_mn2.rename(columns={0: "County", 1: "State Name", "County": "County Code"})
census_pd_mn2["County"] = census_pd_mn2["County"].str.replace("County", '')
del census_pd_mn2["Name"]

census_pd_mn2.head(10)

,Population,state,County Code,County,State Name
1376,15722.0,27,001,Aitkin,Minnesota
1377,341249.0,27,003,Anoka,Minnesota
1378,33375.0,27,005,Becker,Minnesota
1379,45644.0,27,007,Beltrami,Minnesota
1380,39457.0,27,009,Benton,Minnesota
1381,5098.0,27,011,Big Stone,Minnesota
1382,65524.0,27,013,Blue Earth,Minnesota
1383,25327.0,27,015,Brown,Minnesota
1384,35482.0,27,017,Carlton,Minnesota
1385,97143.0,27,019,Carver,Minnesota


In [ ]:
#EPA Air Quality Section
#url = f"https://aqs.epa.gov/data/api/annualData/byState?email=buhler8819@gmail.com&key={epa_key}&param=88101&bdate=20160101&edate=20160101&state=27"

In [ ]:
#response = requests.get(url).json()
#print(json.dumps(response, indent=4, sort_keys=True))

In [48]:
#read in EPA csv files
epa_aqi_2016 = pd.read_csv("AQI_2016.csv")
epa_aqi_2017 = pd.read_csv("AQI_2017.csv")
epa_aqi_2018 = pd.read_csv("AQI_2018.csv")

epa_aqi_mn_2016 = epa_aqi_2016[epa_aqi_2016["State"].str.contains("Minnesota")]
epa_aqi_mn_2017 = epa_aqi_2017[epa_aqi_2017["State"].str.contains("Minnesota")]
epa_aqi_mn_2018 = epa_aqi_2018[epa_aqi_2018["State"].str.contains("Minnesota")]

#epa_aqi_mn_2016 = epa_aqi_mn_2016.rename(columns={"County": "County Name"})

epa_aqi_mn_2016.head(10)

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
461,Minnesota,Anoka,2016,366,328,37,1,0,0,0,145,51,35,0,13,251,0,102,0
462,Minnesota,Becker,2016,361,337,24,0,0,0,0,89,45,27,0,0,197,0,164,0
463,Minnesota,Beltrami,2016,358,344,14,0,0,0,0,71,33,18,0,0,0,0,358,0
464,Minnesota,Carlton,2016,364,345,18,1,0,0,0,101,42,29,0,0,256,0,108,0
465,Minnesota,Cook,2016,356,354,2,0,0,0,0,53,29,14,0,0,0,0,356,0
466,Minnesota,Crow Wing,2016,357,334,23,0,0,0,0,84,45,29,0,0,161,0,196,0
467,Minnesota,Dakota,2016,366,332,32,2,0,0,0,135,50,28,0,90,0,5,271,0
468,Minnesota,Goodhue,2016,214,202,12,0,0,0,0,87,47,34,0,0,214,0,0,0
469,Minnesota,Hennepin,2016,366,237,127,2,0,0,0,145,67,41,0,21,51,0,123,171
470,Minnesota,Lake,2016,366,350,16,0,0,0,0,86,43,22,0,0,171,0,133,62


In [53]:
#census_pd_mn2["County Name"] = census_pd_mn2["County Name"].astype(str)
#epa_aqi_mn_2016["County Name"] = epa_aqi_mn_2016["County Name"].astype(str)

merged_df_2016 = pd.merge(census_pd_mn2, epa_aqi_mn_2016, on = "County", how = "inner")
merged_df_2016.head(10)

,Population,state,County Code,County,State Name,State,Year,Days with AQI,Good Days,Moderate Days,...,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
